In [29]:
# Go through OJK page that contains Statistik Perbankan Indonesia page
# Go through all links of SPI that available in the main SPI page (usually will be the last 10 months)

import os
import requests
import zipfile
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

url = 'https://www.ojk.go.id/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/default.aspx'
root = 'https://www.ojk.go.id/'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

all_hrefs = soup.find_all('a')
all_links = [link.get('href') for link in all_hrefs]

SPI_links = [dl for dl in all_links if dl and 'Statistik-Perbankan-Indonesia---' in dl]

download_folder = 'output/OJK_SPI'

In [36]:
# Go through each SPI links, then download the zip files and unzip them
# unzip the files 

if not os.path.exists(download_folder):
    os.makedirs(download_folder)

tries = 0
for SPI_link in SPI_links:
    url = root + SPI_link
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    all_hrefs = soup.find_all('a')
    all_links = [link.get('href') for link in all_hrefs]
    zip_files = [dl for dl in all_links if dl and '.zip' in dl]

    for zip_file in zip_files:
        full_url = root + zip_file
        zip_filename = os.path.basename(zip_file)
        dl_path = os.path.join(download_folder, zip_filename)
        if os.path.exists(dl_path):
            # you have already downloaded this file, so skip it
            continue

        while tries < 3:
            r = requests.get(full_url)
            dl_path = os.path.join(download_folder, zip_filename)
            with open(dl_path, 'wb') as z_file:
                z_file.write(r.content)

            # unzip the file
            # use this if want to extract the file to subfolder based on file name: extract_dir = os.path.splitext(os.path.basename(zip_file))[0]
            try:
                z = zipfile.ZipFile(dl_path)
                z.extractall(os.path.join(download_folder))
                break
            except zipfile.BadZipfile:
                # the file didn't download correctly, so try again
                # this is also a good place to log the error
                pass
            tries += 1

In [45]:
# Delete all zip files

download_content = os.listdir( download_folder )

for item in test:
    if item.endswith(".zip"):
        os.remove( os.path.join( download_content, item ) )

In [55]:
# Find the latest file

# Calling DataFrame constructor on list 
df_download = pd.DataFrame(download_content, columns =['File_Name']) 
df_download['Name'] = df_download['File_Name'].str.replace('.xlsx', '')

df_download["Month"] = (df_download["Name"].str.split(" ", n = 3, expand = True))[1] 
df_download["Year"] = (df_download["Name"].str.split(" ", n = 3, expand = True))[2] 

conditions = [
              (df_download['Month'] == 'Januari'),
              (df_download['Month'] == 'Februari'),
              (df_download['Month'] == 'Maret'),
              (df_download['Month'] == 'April'),
              (df_download['Month'] == 'Mei'),
              (df_download['Month'] == 'Juni'),
              (df_download['Month'] == 'Juli'),
              (df_download['Month'] == 'Agustus'),
              (df_download['Month'] == 'September'),
              (df_download['Month'] == 'Oktober'),
              (df_download['Month'] == 'November'),
              (df_download['Month'] == 'Desember')
             ]
choices = [
           '1',
           '2',
           '3',
           '4',
           '5',
           '6',
           '7',
           '8',
           '9',
           '10',
           '11',
           '12'
           ]
df_download['Month_str'] = np.select(conditions,choices, default=np.nan) #nan is default value"

df_download['Report_Date'] = pd.to_datetime(df_download['Year'].astype(str)+"-"+df_download['Month_str'].astype(str), format='%Y-%m')

max_date = max(df_download['Report_Date'])
max_report = df_download[df_download['Report_Date'] == max_date]
final_report = max_report.iloc[0]['File_Name']

In [ ]:
Kredit per Lok. Dati I_3.17.a.

